## FBCCA and Extended FBCCA for SSVEP signals classification

In [ ]:
# Imports:
%pip install ffmpeg
from tqdm import tqdm

import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt
import scipy.signal as signal
import scipy.io as sio

import re

from sklearn.cross_decomposition import CCA
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import cohen_kappa_score
from sklearn.metrics import classification_report

from scipy.signal import butter, lfilter, periodogram

import pandas as pd

import os

In [ ]:

df = pd.read_csv('expr/checkpoints/SSVEP_w_2s_log.txt')
"""
Index(['time', 'iteration', 'D/latent_real', 'D/latent_fake', 'D/latent_reg',
       'D/ref_real', 'D/ref_fake', 'D/ref_reg', 'G/latent_adv', 'G/latent_sty',
       'G/latent_ds', 'G/latent_cyc', 'G/ref_adv', 'G/ref_sty', 'G/ref_ds',
       'G/ref_cyc', 'G/lambda_ds'],
      dtype='object')
"""
print(df.head())
# Plots: 
# Disceriminators:
# plt.figure(figsize=(10, 5))
plt.plot(df['iteration'], df['G/ref_sty'], label='Latent')
plt.plot(df['iteration'], df['G/latent_sty'], label='Reference')
plt.title('Style Reconstruction Loss')
plt.legend()
plt.ylim(0, 3)
plt.show()

# # Generators:
# plt.figure(figsize=(10, 5))
# # plt.plot(df['iteration'], df['G/lambda_ds'], label='G/lambda_ds')
# plt.plot(df['iteration'], df['D/ref_fake'], label='G/ref_fake')
# plt.plot(df['iteration'], df['D/ref_real'], label='G/latent_real')
# # plt.plot(df['iteration'], df['G/ref_ds'], label='G/ref_ds')
# plt.ylim(0, 3)
# plt.title('Generator Loss')
# plt.legend()
# plt.show()



In [68]:
# get the src data for the model: 
os.makedirs('data/colorado/processed', exist_ok=True)

subjects = ['s1_trial1.csv', 's2_trial2.csv', 's3_trial3.csv', 's4_trial4.csv']
for i, subject in enumerate(subjects):
	df = pd.read_csv(f'data/colorado/processed/{subject}')

	os.makedirs(f'data/colorado/test/{i}', exist_ok=True)

	segment = df.iloc[:1024].values.swapaxes(0, 1)
	np.save(f'data/colorado/test/{i}/1024_segment.npy', segment)

In [1]:
from os import makedirs


class Args:
    img_size = 1024
    num_domains = 4
    latent_dim = 16
    hidden_dim = 512
    style_dim = 64
    lambda_reg = 1
    lambda_cyc = 1
    lambda_sty = 1
    lambda_ds = 1
    ds_iter = 5000
    w_hpf = 0  # For SSVEP
    randcrop_prob = 0.5
    total_iters = 5000
    resume_iter = 0
    batch_size = 8
    val_batch_size = 8
    lr = 5e-4
    f_lr = 1e-6
    beta1 = 0.0
    beta2 = 0.99
    weight_decay = 1e-4
    num_outs_per_domain = 2
    mode = 'train'
    num_workers = 16
    seed = 777
    train_img_dir = 'data/train'
    val_img_dir = 'data/val'
    # sample_dir = 'expr/samples'
    checkpoint_dir = 'expr/checkpoints'
    eval_dir = 'expr/eval'
    result_dir = 'expr/results'
    print_every = 10
    sample_every = 250
    save_every = 1000
    eval_every = 2500
    runType = f'w_4s'
    sample_dir = f'expr/samples/{runType}'

args = Args()
args.skip_data = False  # Add this line to initialize skip_data attribute

makedirs(args.sample_dir, exist_ok=True)

In [2]:
SRC_DIR = "data/colorado/test/"
REF_DIR = "data/ref/"
NUM_DOMAINS = 4
IMG_SIZE = 1024
VAL_BATCH_SIZE = 4
NUM_WORKERS = 4

import os
from munch import Munch
from core.data_loader import get_test_loader
from core.solver import Solver

def subdirs(dname):
    return [d for d in os.listdir(dname)
            if os.path.isdir(os.path.join(dname, d))]

# assert len(subdirs(SRC_DIR)) == NUM_DOMAINS?
assert len(subdirs(REF_DIR)) == NUM_DOMAINS

solver = Solver(args)

loaders = Munch(src=get_test_loader(root=SRC_DIR,
                                    img_size=IMG_SIZE,
                                    batch_size=VAL_BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS),
                ref=get_test_loader(root=REF_DIR,
                                    img_size=IMG_SIZE,
                                    batch_size=VAL_BATCH_SIZE,
                                    shuffle=False,
                                    num_workers=NUM_WORKERS))
solver.sample(loaders)

Training on cuda
Number of parameters of generator: 12161329
Number of parameters of mapping_network: 4079872
Number of parameters of style_encoder: 7008622
Number of parameters of discriminator: 6879346
Initializing generator...
Initializing mapping_network...
Initializing style_encoder...
Initializing discriminator...
Preparing DataLoader for the generation phase...
Preparing DataLoader for the generation phase...
Loading checkpoint from expr/checkpoints/SSVEP_w_4s_nets.ckpt...


/home/loaspra/stargan-ssvep/core/checkpoint.py:42: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  module_dict = torch.load(fname)


Loading checkpoint from expr/checkpoints/SSVEP_w_4s_nets_ema.ckpt...
Loading checkpoint from expr/checkpoints/SSVEP_w_4s_optims.ckpt...
Working on latent-guided image synthesis...
Translating using latent with x_src shape: torch.Size([4, 3, 1024])
Saving latent into expr/samples/w_4s/latent_psi_0.5.jpg_latent_0.png
Saving latent into expr/samples/w_4s/latent_psi_0.5.jpg_latent_1.png
Saving latent into expr/samples/w_4s/latent_psi_0.5.jpg_latent_2.png
Saving latent into expr/samples/w_4s/latent_psi_0.5.jpg_latent_3.png
Translating using latent with x_src shape: torch.Size([4, 3, 1024])
Saving latent into expr/samples/w_4s/latent_psi_0.7.jpg_latent_0.png
Saving latent into expr/samples/w_4s/latent_psi_0.7.jpg_latent_1.png
Saving latent into expr/samples/w_4s/latent_psi_0.7.jpg_latent_2.png
Saving latent into expr/samples/w_4s/latent_psi_0.7.jpg_latent_3.png
Translating using latent with x_src shape: torch.Size([4, 3, 1024])
Saving latent into expr/samples/w_4s/latent_psi_1.0.jpg_latent_0

In [ ]:
from core.utils2 import generate_ref_signal, combined_cca

# SSVEP target frequencies
frequencies = [8, 10, 12, 14]
# Number of samples
N = 1024
# Sampling frequency
fs = 250
# Number of harmonics
n_harmonics = 3
# Number of sub-bands
n_subbands = 8
# Filter bank design
filter_bank_design = 'M1'
# Reference signals
ref_signals = generate_ref_signal(
    'data/raw/Freq_Phase.mat', frequencies, N, n_harmonics, fs)

# combined_cca

In [ ]:
def get_acc_of_subject_ECCA(subject, path, template_path, target, labels, synt_ratio=0.5):

    synth_template = np.zeros((4, 6, 3, 1024))
    # leave 1 window to perform testing (load 6 out of 7 windows to create the template)
    real_template = np.zeros((4, 6, 3, 1024))

    # Get the synthetic data for the subject (results of StarGan)
    for label in labels:
        synth_template[label] = np.load(f"{template_path}ref_0{label}_b.npy")[
            :6, :, :]  # (8, 3, 1024) 8 is batch_size
        # Get the real data for the subject
        for i in range(6):
            real_template[label, i, :, :] = np.load(
                path + f"{label}/S{subject}_{i}.npy")

    # get the average of the concatenated array
    if synt_ratio == 0:
        template = np.mean(real_template, axis=1)
    elif synt_ratio == 1:
        template = np.mean(synth_template, axis=1)
    else:
        template = np.mean(
            np.concatenate((
                real_template[:,:(int((1 - synt_ratio) * synth_template.shape[0]))],
                synth_template[:,:int(synt_ratio) *
                                   synth_template.shape[0]]
            ), axis = 1), axis=1)

    in_signal = np.load(path + f"{target}/S{subject}_6.npy")
    # print(f"Shape of templates: {real_template.shape}, {synth_template.shape}, {template.shape}")
    target_frequency = combined_cca(
        in_signal, template, fs, n_subbands, filter_bank_design, ref_signals, 8)

    acc = accuracy_score([target], [target_frequency])

    return acc

def get_acc_of_subject(subject, path, label, w_size=4):

    # get all .npy files of the subject
    files = os.listdir(path)

    predictions = []

    for file in files:
        if re.match(f"^{subject}_.\.npy$", file) and file.endswith(".npy"):
            in_signal = np.load(path + file)
            target_frequency = fbcca(
                in_signal, fs, n_subbands, filter_bank_design, ref_signals, 8)
            predictions.append(target_frequency)

    targets = np.ones(len(predictions)) * label
    # calculate the accuracy
    acc = accuracy_score(targets, predictions)

    return acc


In [ ]:
data = np.load("expr/results/ref_03_w_4s.npy")
data.shape

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np
IDX_FREQS = [0, 2, 4, 6]  # [8, 10, 12, 14]

file_path = "data/train/0/S4_2.npy"
segment = np.load(file_path) # .swapaxes(0, 1)

print(data.shape)

accuracies = np.zeros((36, 6))

dominant_freqs = []
# print(segment.shape)
for channel in segment:
    freqs, psd = signal.welch(channel, fs=250, nperseg=250)
    dominant_freq = np.argmax(psd)
    print(dominant_freq)
    dominant_freqs.append(dominant_freq)
if dominant_freqs.count(8) > 1:
    accuracies[3, 2] = 1
else:
    accuracies[3, 2] = 0

print(accuracies)
dominant_freqs


In [ ]:
files = ["S1_0.npy", "S4_2.npy", "S33_1.npy", "S33_2.npy", "S33_3.npy", "S33_4.npy", "S33_5.npy", "S33_4.npy"]
files = [file for file in files if accuracies[int(file.split("_")[0][1:]) - 1, int(file.split("_")[1][0:1])] == 1]
print(files)

In [ ]:
import matplotlib.pyplot as plt
from scipy import signal
import numpy as np


data = np.load("data/train/0/S4_2.npy")
print(data.shape)

# plot the data
sample = data[0, :] # (3, 1024)
# data[3, 1, 2, :] --> 12 Hz


# plt.figure(figsize=(10, 5))
# plt.plot(data)
# plt.show()

freqs, psd = signal.welch(sample, fs=250, nperseg=250)
# set all negatives to 0
plt.figure(figsize=(12, 6))
plt.stem(freqs, psd)

plt.xlabel('Frequency (Hz)')
plt.ylabel('Power/Frequency (dB/Hz)')
plt.title('Power Spectral Density of EEG Signal')
plt.xlim(0, 50)
plt.show()



# plot the psd of the signal

In [ ]:
################ Windows size = 4 seconds
acc1_4 = [0.800, 0.810, 0.813, 0.820, 0.827]
std1_4 = [0.01, 0.015, 0.012, 0.013, 0.014]

# Accuracies using synthetic data
acc2_4 = [0.750, 0.760, 0.763, 0.770, 0.777]
std2_4 = [0.02, 0.018, 0.017, 0.019, 0.019]

################ Windows size = 2 seconds
acc1_2 = [0.68, 0.725, 0.71, 0.712, 0.725]
std1_2 = [0.03, 0.02, 0.03, 0.019, 0.023]

# Accuracies using synthetic data
acc2_2 = [0.583, 0.625, 0.62, 0.66, 0.66]
std2_2 = [0.032, 0.02, 0.03, 0.019, 0.019]

################ Windows size = 1 seconds
acc1_1 = [0.66, 0.66, 0.66, 0.725, 0.62]
std1_1 = [0.03, 0.02, 0.03, 0.019, 0.023]

# Accuracies using synthetic data
acc2_1 = [0.66, 0.5, 0.625, 0.705, 0.5]
std2_1 = [0.032, 0.02, 0.03, 0.019, 0.019]

# Define the dataframe for all the window sizes
df_1 = pd.DataFrame({
    'Samples': list(range(1, 6)) * 2,
    'Accuracy': acc1_1 + acc2_1,
    'Data': ['Real'] * 5 + ['Synthetic'] * 5,
    'Std': std1_1 + std2_1,
    'Window': ['1s'] * 10
})

df_2 = pd.DataFrame({
    'Samples': list(range(1, 6)) * 2,
    'Accuracy': acc1_2 + acc2_2,
    'Data': ['Real'] * 5 + ['Synthetic'] * 5,
    'Std': std1_2 + std2_2,
    'Window': ['2s'] * 10
})

df_4 = pd.DataFrame({
    'Samples': list(range(1, 6)) * 2,
    'Accuracy': acc1_4 + acc2_4,
    'Data': ['Real'] * 5 + ['Synthetic'] * 5,
    'Std': std1_4 + std2_4,
    'Window': ['4s'] * 10
})



In [ ]:
latent_sig = "expr/samples/w_4s/latent_psi_1.0.jpg_w_4s.npy"

data = np.load(latent_sig)
print(data.shape)

sample = data[2, 2, :]
# data[2, :, :] --> 8hz
# data[3 y 4 y 7, :] --> 10hz
# data[5 y 6, 2, :] --> 12hz
# data[1, 2, :] --> 14hz

In [ ]:
template_path = "expr/results/vanilla/"
path = "data/staged/"
labels = [0, 1, 2, 3]

bins = 4
accs_ratio = np.zeros(bins + 1)

for k in range(bins + 1):

    accs = np.zeros(36)

    for label in tqdm(labels):

        path = f"data/staged/"

        for i in range(1, 36):
            acc = get_acc_of_subject_ECCA(
                i, path, template_path, label, labels, synt_ratio=(1/bins) * k)
            accs[i - 1] += acc * 1 / len(labels)

    accs_ratio[k] = np.mean(accs)

print(accs_ratio)

In [ ]:
labels = [0, 1, 2, 3]

window_size = 4

accs = np.zeros(36)

for label in tqdm(labels):

    path = f"data/staged/{label}/"

    for i in range(1, 36):
        acc = get_acc_of_subject(f"S{i}", path, label, window_size)
        accs[i - 1] += acc * 1 / len(labels)


print(accs)


In [ ]:
FREQS = [8, 10, 12, 14]
IDX_FREQS = [0, 2, 4, 6]
for i, (freq, idX_fre) in enumerate(zip(FREQS, IDX_FREQS)):
    print(i, freq, idX_fre)